# Analysis of Steinmetz Neuropixels recordings of spontaneous activity in mice (Week 11-12)
---
This notebook contains consolidated materials from week 1 to 10, consisting of the following sections:
- Overview of dataset
- Correlation-matrix-based Hierarchical Clustering

In [ ]:
using Plots, Measures; gr()
using MAT, Printf
using LinearAlgebra, Statistics, Random
using Clustering, Distances
using InformationMeasures
using BenchmarkTools

## Overview of dataset

### Backgrounds
The dataset in its original form contains dataset types consistent with the *Open Neurophysiology Environment* (ONE) standard, which is part of a system proposed by the International Brain Laboratory (IBL) and colleagues with the aim of facilitating large-scale collaboration in neuroscience. Specifically, the main data consists of eight-probe Neuropixels recordings of spontaneous activity in three mice in the following format:
- spks(k).st = time of spikes in seconds
- spks(k).clu = neuron ID associated with each spike in st
- spks(k).Wheights = height of each neuron on the probe [IS THIS `Wheights` or `heights`?

Note: Data from each mouse is stored in a **spks** which is a matlab structure with eight immediate children each corresponding to a probe. Therefore , $k$ ranges from 1 to 8. Supplementary data, such as labeling of the brain area based on location and information about the probes are also included. The following is an illustration of the probes inside a wire-mesh brain using location information of the probes in the Allen CCF framework. The probes have one of three colors, and each color is associated with one of the three mice.

![alt](./Plots/meshbrain.png)

In its original form, the data does not allow for considerable freedom in analysis and, fortunately, a matlab file for preprocessing the data is included. [WHAT DO YOU MEAN BY "DOES NOT ALLOW FOR CONSIDERABLE FREEDOM OF ANALYSIS"]  After executing the code, the data are stored in 1D and 2D arrays each assigned to a variable. [AREN'T ARRAYS ALWAYS ASSIGNED TO A VARIABLE? WHAT IS THE ALTERNATIVE? DO YOU MEAN "VARIABLE NAME", A "TAG", A "LABEL"? YOU MUST BE AS PRECISE AS POSSIBLE.] Listing and details of these variables are provided in the next section.

### Components
The processed data for each of the three mice are stored in a `.mat` file, containing the following variables:
- `areaLabels` string array of abbreviated names of different brain areas
- `brainLoc` array of indexes corresponding to brain areas in `areaLabels` (one for each neuron) [TO BE CLEAR: IF THERE ARE 5 BRAIN AREAS, EACH NEURON HAS A NUMBER FROM 0 TO 4? DO THE DIFFERENT MICE HAVE THE SAME NUMBER OF BRAIN AREAS AND ARE THEY NUMBERED THE SAME WAY? 
- `iprobe` array of probes IDs (one for each neuron)  [IS THIS A SINGLE ARRAY ACROSS ALL MICE OR ONE ARRAY PER MOUSE. YOU ARE NOT CLEAR ABOUT THIS]
- `mostSVD` svd components of movie frames of mouse face during recording period  [WHY `mostSVD` AS OPPOSED TO `SVD`? IS THIS JUST THE DOMINANT SVDS? YOU SEE ALL THE QUESTIONS I ASK?]
- `srate` sampling rate (30Hz)
- `stall` matrix containing spike trains of all neurons [WHAT ARE DIMENSIONS OF THIS MATRIX? M[I,J]. WHAT IS I AND WHAT IS J? WHAT ARE MAXIMUM AND MINIMUM VALUES? WHAT DOES THE VALUE OF M[I,J] REPRESENT?]
- `tspont` array of sample timestamps
- `tVid` times of movie frames in spike reference timeframe [NOT SURE WHAT THIS MEANS]
- `Wh` array of relative heights of the neurons [WHAT IS MEANT BY `RELATIVE HEIGHTS`?

[YOU SHOULD ANSWER ALL THE QUESTIONS I ASK IN A PARAGRAPH FOLLOWING THE LIST OF VARIABLES]
The variables are loaded into THE workspace as elements of a dictionary using the `MAT` package as shown below.

In [ ]:
# Load all data in a three-element array of dictionaries
# Each element corresponding to each mouse maps variable names to their contents  
mice_names = ["Krebs", "Waksman", "Robbins"]
data = [MAT.matread("./Data/$(mice_names[i])withFaces_KS2.mat") for i in 1:3];

The following cells illustrate the spiking activity of neurons in each mouse. Note that the heatmaps were modified to only show whether spikes were detected: dark and bright pixels indicate the absence and presence of spikes. Although a sample for a neuron (an element in the spike train matrix) can have values greater than one, this is relatively rare. These values are sometimes high enough that the unaltered heatmap looks almost uniformly dark, which defeats the purpose of the illustrations. One option is to make the heatmap "binary", showing presence and absence of spikes and, as a result, one can easily see that the former is dominant in all three cases. Also of note is that we scale the x and y axes of the heatmaps based on the number of neurons involved and the length of the recording period to show the difference in size of data between the mice. Therefore, the range of the $x$ and $y$ axes are the same for all three mice. [I ADDED THIS SENTENCE. IS IT CORRECT? IF NOT, THAT MEANS I DO NOT UNDERSTAND YOUR COMMENT ABOUT SCALING THE X AND Y AXES OF THE HEATMAPS.]

In [ ]:
# Run the following code to demonstrate that heatmap does not generate a bindary: 

function testHeatmap()
	h = Random.randn(20,20)	
	println("min(h): ", minimum(h))
	println("max(h): ", maximum(h))
	println()
	display(heatmap(h,clim=(0,1), colorbar=false))
end

testHeatmap()

In [ ]:
function spkTrainShow(data, names, id)
    st_all = data[id]["stall"]
    n_neuron, n_sample = size(st_all)  # GE: strictly speaking, it should be n_neurons, n_samples
    display(heatmap(st_all, clim=(0, 1), yticks=0:500:3000, yflip=true, colorbar=false, 
            size=(round(n_sample/5000)*100, round(n_neuron/10)), bottom_margin=5mm))
    println("Name: $(names[id])\tID: $id")
    println("Number of neurons: $n_neuron")
    println("Number of samples (length of recording x sampling rate): $n_sample")
    println("Largest number of spikes of a neuron in a sample: $(maximum(st_all))")
end
# GE: I am not convinced that your heatmap has only two colors. 
# This would only be the case if `st_all` had only two possible values. 

In [ ]:
spkTrainShow(data, mice_names, 1)

In [ ]:
spkTrainShow(data, mice_names, 2)

In [ ]:
spkTrainShow(data, mice_names, 3)

The next few cells include plots of the heights of the neurons on individual probes to provide a sense of the distance between the neurons. This becomes relevant in the analysis portion of this notebook.
[MAKE SURE YOU HAVE A SUBTITLE CALLED "ANALYSIS" SO THAT THE READER SEES THE LINK BETWEEN WHAT YOU JUST WROTE AND THE ANALYSIS PORTION OF THE NOTEBOOK.]

In [ ]:
""" Add DocStrings ahead of every function 
 https://docs.julialang.org/en/v1/manual/documentation/ 
Document the meaning of the arguments. Get into the habit.
If `id` refers to the mouse id, replace `id` by `mouse_id` """
function hobShow(data, id)
    plot_array = Array{Plots.Plot{Plots.GRBackend}}(undef, 8)
    probe_ids = data[id]["iprobe"]
    hob_all = data[id]["Wh"]
    
    for i=1:8
        hob_iprobe = hob_all[vec(probe_ids .== i)]
        plot_array[i] = scatter(hob_iprobe, ylim=(0, 4000), xlabel="Neuron", ylabel="Height (μm)", title="M$id-P$i", legend=false, markersize=1)
    end
 b   
    display(plot(plot_array..., layout=(4, 2), size=(1250, 1800), left_margin=8mm))
end

In [ ]:
hobShow(data, 1)

In [ ]:
hobShow(data, 2)

In [ ]:
hobShow(data, 3)

## Correlation-matrix-based Hierarchical Clustering

### Comparison of correlation to other similarity/distance measures

[GE] Describe the arguments of the `spkCorCompare` function using """docString""" in front of the function. `link` and `id` are not obvious. Here is a version easier to understand.

[GE] What is the difference between `byprobe=false` or `true`? It is either one matrix per probe versus one matrix per mouse, correct. That means that before computing the correlation, mutual information, distances, the only thing that changes is the matrix size and content. Therefore, the code is easier to understand if you have three functions: `distance`, `correlation`, `mutualInformation` that take as input, matrix and other quantities associated with the matrix. Then all you do in your code is something like: 

```if byprobe
   setup matrices for byprobe
   for i in probes
      distance()
      correlation()
      mutualInformation()
      plotData()
   end
else
   setup matrices for this case
   distance()
   correlation()
   mutualInformation()
   plotData()
end```

The code above is simpler to understand and is self-documented. It is up to you how to choose the function arguments for clarity. Smaller functions promote reusability. This code is also easier to debug. Finally, it is better to keep all information related to mutual information in one place, all information related to correlation in one place, etc. In C++ or Python, you'd create classes for Correlation, MutualInformation, etc. 

In [ ]:
function spkCorCompare(data, id, link, metric_sym=:None, byprobe=false)
    st_all = data[id]["stall"]      # muti-neuron spike train matrix
    hop_all = data[id]["Wh"]        # relative height of each neuron on a probe
    
    n_neuron = size(st_all, 1)
    sym2func = Dict(:CosD => Distances.CosineDist(), :EucD => Distances.Euclidean())
    
    if byprobe
        probe_ids = data[id]["iprobe"]        # array of corresponding probe IDs
        plot_array = Array{Plots.Plot{Plots.GRBackend}}(undef, 8*3)
        
        for i=1:8
            extract_ids = vec(probe_ids .== i)
            st_iprobe = st_all[extract_ids, :]
            n_neuron_iprobe = size(st_iprobe, 1)
            
            # proximity
            hop_iprobe = hop_all[extract_ids]
            n_neuron_iprobe = length(hop_iprobe)
            rep_mat = repeat(hop_iprobe, 1, n_neuron_iprobe)
            dist_mat = abs.(rep_mat - transpose(rep_mat)) ./ 4000
            
            # correlation
            cor_mat = Statistics.cor(st_iprobe, dims=2)
            if metric_sym != :None
                diss_mat = Distances.pairwise(sym2func[metric_sym], cor_mat, dims=1)
                result = Clustering.hclust(diss_mat, linkage=link)
                clust_idx = result.order
            else
                clust_idx = 1:n_neuron_iprobe
            end
            
            # mutual information
            mi_mat = zeros(n_neuron_iprobe, n_neuron_iprobe)
            for i=1:n_neuron_iprobe-1
                for j=i+1:n_neuron_iprobe
                    mi_mat[i, j] = InformationMeasures.get_mutual_information(st_iprobe[i, :], st_iprobe[j, :])
                end
            end

            mi_mat = mi_mat + transpose(mi_mat)

            for i=1:n_neuron_iprobe
                mi_mat[i, i] = InformationMeasures.get_entropy(st_iprobe[i, :])
            end
                
            mi_mat = log.(mi_mat)

            # store plots in array
            plot_array[i*3-2] = heatmap(dist_mat[clust_idx, clust_idx], clim=(0, 1), title="M$id-P$i-ndist-$metric_sym")
            plot_array[i*3-1] = heatmap(cor_mat[clust_idx, clust_idx], clim=(-0.3, 1), title="M$id-P$i-cor-$metric_sym")
            plot_array[i*3] = heatmap(mi_mat[clust_idx, clust_idx], clim=(-20, 1.1), title="M$id-P$i-minfo-$metric_sym")
        end 
        
        display(plot(plot_array[1:12]..., layout=(4, 3), size=(1500, 1500), yflip=true))
        #savefig("./plots/corCompare_p1-4_m$(id)($metric_sym).png")
        display(plot(plot_array[13:24]..., layout=(4, 3), size=(1500, 1500), yflip=true))
        #savefig("./plots/corCompare_p5-8_m$(id)($metric_sym).png")
    else
        # correlation
        cor_mat = Statistics.cor(st_all, dims=2)
        if metric_sym != :None
            diss_mat = Distances.pairwise(sym2func[metric_sym], cor_mat, dims=1)
            result = Clustering.hclust(diss_mat, linkage=link)
            clust_idx = result.order
        else
            clust_idx = 1:n_neuron
        end
            
        # proximity
        rep_mat = repeat(hop_all, 1, n_neuron)
        dist_mat = abs.(rep_mat - transpose(rep_mat))
        norm_dist_mat = dist_mat ./ maximum(dist_mat)
        
        # mutual information
        mi_mat = zeros(n_neuron, n_neuron)
        for i=1:n_neuron-1
            for j=i+1:n_neuron
                compare_mat[i, j] = InformationMeasures.get_mutual_information(st_all[i, :], st_all[j, :])
            end
        end
            
        compare_mat = compare_mat + transpose(compare_mat)
        
        for i=1:n_neuron_iprobe
            mi_mat[i, i] = InformationMeasures.get_entropy(st_iprobe[i, :])
        end
                
        mi_mat = log.(mi_mat)
        
        p1 = heatmap(norm_dist_mat[clust_idx, clust_idx], title="M$id-all-ndist-$metric_sym")
        p2 = heatmap(cor_mat[clust_idx, clust_idx], clim=(-0.3, 1), title="M$id-all-cor-$metric_sym")
        p3 = heatmap(mi_mat[clust_idx, clust_idx], clim=(-20, 1.1), title="M$id-all-minfo-$metric_sym")
        display(plot(p1, p2, layout=(1, 3), size=(1500, 450), yflip=true))
        #savefig("./plots/corVS$(comparison)_all_m$(id)($str).png")
    end
    
end

In [ ]:
# [GE] code is clear if you put the name of the argument: metric_sym in this case
spkCorCompare(data, 1, :average, metric_sym:CosD, byprobe=true)

In [ ]:
spkCorCompare(data, 2, :average, :CosD, true)

In [ ]:
spkCorCompare(data, 3, :average, :CosD, true)

### Cluster Analysis 

In [ ]:
function spkCorClust(data, id)
    st_all = data[id]["stall"]
    probe_ids = data[id]["iprobe"]
    loc_all = data[id]["brainLoc"]
    
    link = :average
    plot_array = Array{Plots.Plot{Plots.GRBackend}}(undef, 8*3)
    
    for i=1:8
        extract_idx = vec(probe_ids .== i)
        
        # group by brain location
        loc_iprobe   = loc_all[extract_idx]
        sort_idx_loc = sortperm(loc_iprobe)
        
        # group by distance measures
        st_iprobe    = st_all[extract_idx, :]
        cor_mat      = Statistics.cor(st_iprobe, dims=2)
        diss_mat     = Distances.pairwise(Distances.CosineDist(), cor_mat, dims=1)
        # [GE] What is `linkage`? You should add some comments. What did you choose :average for `link`? Do 
        # other choices makes a difference?
        result       = Clustering.hclust(diss_mat, linkage=link)
        sort_idx_cos = result.order
        diss_mat     = Distances.pairwise(Distances.Euclidean(), cor_mat, dims=1)
        result       = Clustering.hclust(diss_mat, linkage=link)
        sort_idx_euc = result.order
        
        # store plots in array  [GE: I aligned the equal signs for better readability. Then, lack of alignment
        # of the resulting expressions makes it easier to find bugs. 
        plot_array[i*3-2] = heatmap(cor_mat[sort_idx_loc, sort_idx_loc], title="M$id-P$i-cor-bLoc")
        plot_array[i*3-1] = heatmap(cor_mat[sort_idx_cos, sort_idx_cos], title="M$id-P$i-cor-CosD")
        plot_array[i*3]   = heatmap(cor_mat[sort_idx_euc, sort_idx_euc], title="M$id-P$i-cor-EucD")
    end
    
    display(plot(plot_array[1:12]..., layout=(4, 3), size=(1500, 1500), clim=(-0.3, 1), yflip=true))
    #savefig("./plots/corVS$(comparison)_p1-4_m$(id)($str).png")
    display(plot(plot_array[13:24]..., layout=(4, 3), size=(1500, 1500), clim=(-0.3, 1), yflip=true))
    #savefig("./plots/corVS$(comparison)_p5-8_m$(id)($str).png")
end

In [ ]:
spkCorClust(data, 1)

In [ ]:
spkCorClust(data, 2)

In [ ]:
spkCorClust(data, 3)

AFTER EACH PLOT, ADD A COMMENT SECTION WHERE YOU DESCRIBE WHAT THE PLOTS TEACH YOU AND WHAT CONCLUSIONS YOU CAN DRAW .